# FABRIC Experiment

FABRIC provides a pair of layer 3 IP networking services across every FABRIC site (FABnetv4 and FABnetv6). You can think of this service as a private internet that connects experiments across the testbed using FABRIC's high-performance network links. 

This notebook describes how to use the FABnetv4 service which is FABRIC's private IPv4 internet.   


In [ ]:
import os
import sys
import time
from datetime import datetime
from dateutil import tz


module_path = os.path.abspath(os.path.join(f"{os.environ['HOME']}/work/PRUTH-FABRIC-Examples/fablib_local"))
if module_path not in sys.path:
    sys.path.append(module_path)
from fablib_custom.fablib_custom import *

from my_experiment.iperf3_fabnet_site_pair import MyExperiment

#fablib = fablib_manager(output_type='HTML')

In [ ]:
#run_name=f'iperf3_test_{time_stamp}'
#print(f"run_name: {run_name}")
#sites=['MICH','NCSA', 'DALL','TACC','UTAH','SALT','STAR','WASH','MASS','MAX']
#sites=['MICH', 'DALL','TACC','UTAH','SALT','STAR','WASH','MASS','MAX']
sites=['UTAH','DALL']

my_experiment = MyExperiment('iperf3_test',
                             output_type='HTML',
                             sites=sites,
                             node_cores=16, 
                             node_ram=64, 
                             node_disk=100, 
                             nic_type='NIC_Basic',
                             node_tools_dir=f"{os.environ['HOME']}/work/PRUTH-FABRIC-Examples/fabric_node_tools")



In [ ]:
my_experiment.deploy()
#my_experiment.load(run_name)

In [ ]:
my_experiment.configure()

In [ ]:
from IPython.display import clear_output
from concurrent.futures import ThreadPoolExecutor

thread_pool_executor = ThreadPoolExecutor(1)
    

runs = [ 
        ('UTAH','DALL'), ('DALL','UTAH'),   
        #('SALT','UTAH'), ('UTAH','SALT'),
        #('SALT','STAR'), ('STAR','SALT'),
        #('SALT','DALL'), ('DALL','SALT'),   
        #('DALL','STAR'), ('STAR','DALL'),
        #('WASH','STAR'), ('STAR','WASH'),
        #('WASH','DALL'), ('DALL','WASH'),
        #('NCSA','STAR'), ('STAR','NCSA'),
        #('MICH','STAR'), ('STAR','MICH'),
        ]
    
threads = {}
for run in runs:
    threads[run] = thread_pool_executor.submit(my_experiment.run,runs=[run], 
                 O=10, w='32M', P=8, t=20, i=10) 
   
 
while len(threads) > 0:
    for run,thread in threads.items():
        if thread.done():
            del threads[run]
            clear_output(wait=True)
            my_experiment.results()
            break
    time.sleep(10)
print(f"done!")
#my_experiment.run(run_name=f'run1', runs=runs, 
#                  O=10, w='32M', P=8, t=70, i=10)
clear_output(wait=True)
my_experiment.results()

In [ ]:
my_experiment.clean_up()